In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"

s3_bucket = "s3://mist-secorapp-{env}/rrm-local/rrm-local-{env}/".format(env=env)

date_day = "2020-03-0*"
hr = '*'

rrm_local_path = "dt={day}/hr={hr}/*.seq".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)



user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-secorapp-production/rrm-local/rrm-local-production/dt=2020-03-0*/hr=*/*.seq


In [2]:
rdd_rrm_local = spark.sparkContext.sequenceFile(rrm_local_path)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.sequenceFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): java.io.IOException: Failed to connect to ip-10-2-17-46.ec2.internal:57154
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:245)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:187)
	at org.apache.spark.rpc.netty.NettyRpcEnv.org$apache$spark$rpc$netty$NettyRpcEnv$$downloadClient(NettyRpcEnv.scala:368)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply$mcV$sp(NettyRpcEnv.scala:336)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.apache.spark.rpc.netty.NettyRpcEnv$$anonfun$openChannel$1.apply(NettyRpcEnv.scala:335)
	at org.apa

In [ ]:
rdd_rrm_local.first()

In [ ]:
df_rrm = rdd_rrm_local.map(lambda x: json.loads(x[1])).toDF()
df_rrm.show()

In [ ]:
df_USNYELE = df_rrm.filter((col('site_id')==user_site_USNYELE) & (col('band')==5))
df_USFLLON = df_rrm.filter((col('site_id')==user_site_USFLLON) & (col('band')==5))

In [ ]:
df_USNYELE.select("command", "reason").groupBy("command", 'reason').count().show()

In [ ]:
df_USFLLON.select("command", "reason").groupBy("command", 'reason').count().show()

In [ ]:
df_USFLLON.filter(col("command")=="rrm-global").groupBy("band",'when', "command", 'reason').count().show()